# Menganalisis Tanda Terima dengan Form Recognizer

![Robot memegang tanda terima](./images/receipt_analysis.jpg)

Di bidang kecerdasan buatan (AI) pada visi komputer, pengenalan karakter optik (OCR) umumnya digunakan untuk membaca dokumen cetak atau tulisan tangan. Seringkali, teks hanya diekstrak dari dokumen ke dalam format yang dapat digunakan untuk pemrosesan atau analisis lebih lanjut.

Skenario OCR yang lebih canggih adalah ekstraksi informasi dari formulir, seperti pesanan atau faktur pembelian, dengan pemahaman semantik tentang apa yang disajikan bidang dalam formulir. Layanan **Form Recognizer** secara spesifik didesain untuk masalah AI jenis ini.

## Melihat tanda terima

Dalam contoh ini, Anda akan menggunakan model bawaan Form Recognizer untuk menganalisis tanda terima.

Klik tombol **Jalankan sel** (&#9655;) (di sebelah kiri sel) di bawah untuk menjalankannya dan melihat contoh tanda terima yang akan digunakan untuk menganalisis Form Recognizer.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Membuat sumber daya Form Recognizer

>**Catatan:** Anda dapat menggunakan sumber daya Cognitive Services atau sumber daya Form Recognizer untuk mengakses layanan Form Recognizer. 

Untuk membuat sumber daya Form Recognizer di langganan Azure Anda:

1. Di tab browser lain, buka portal Microsoft Azure di https://portal.azure.com, masuk menggunakan akun Microsoft Anda.
2. Pilih **+ Buat sumber daya**, dan cari *Form Recognizer*.
3. Dalam daftar layanan, pilih **Form Recognizer**.
4. Pada blade **Form Recognizer**, pilih **Buat**.
5. Pada bilah **Buat**, masukkan detail berikut dan pilih **Buat**
   - **Nama**: Nama unik untuk layanan Anda
   - **Langganan**: Langganan Azure Anda
   - **Wilayah**: Wilayah yang tersedia
   - **Tingkat Harga**: F0
   - **Grup Sumber Daya**: Grup sumber daya yang ada yang Anda gunakan sebelumnya
   - **Saya mengonfirmasi bahwa saya telah membaca dan memahami pemberitahuan di bawah**: Dipilih.
6. Tunggu hingga layanan dibuat.
7. Lihat layanan Form Recognizer yang baru dibuat di portal Microsoft Azure dan di halaman **Kunci dan Titik Akhir**, salin nilai **Key1** dan **Titik Akhir** lalu tempel pada sel kode di bawah, menggantikan **YOUR_FORM_KEY** dan **YOUR_FORM_ENDPOINT**.

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## Menganalisis tanda terima

Sekarang Anda siap menggunakan Form Recognizer untuk menganalisis tanda terima.

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Perhatikan bahwa Form Recognizer dapat menafsirkan data dalam formulir, mengidentifikasi alamat dan nomor telepon penjual, serta tanggal dan waktu transaksi, juga item baris, subtotal, pajak, dan jumlah total dengan benar.

## Informasi Selengkapnya

Untuk informasi selengkapnya tentang layanan Form Recognizer, lihat [dokumentasi Form Recognizer](https://docs.microsoft.com/id-id/azure/cognitive-services/form-recognizer/index)